In [1]:
import folium
import ftplib
from folium import plugins
import pandas as pd
import requests
from urllib.parse import urljoin
import ftplib
host = 'data-lab.kremppi.com'
ftp_user = ''
ftp_password = ''
username = ''
token = ''
pythonanywhere_host = "www.pythonanywhere.com"
api_base = "https://{pythonanywhere_host}/api/v0/user/{username}/".format(
    pythonanywhere_host=pythonanywhere_host,
    username=username,
)

In [21]:
resp = requests.get(urljoin(api_base, "files/path/home/{username}/log.csv".format(username=username)), headers = {"Authorization": "Token {api_token}".format(api_token=token)})

with open('log.csv', 'wb') as f:
    f.write(resp.content)

data = pd.read_csv("log.csv", sep='&')

data



,date%time,lat,long,temp,windvel,winddir,humidity,pressure,description
0,2020-02-13 17:45:29+00,46.345354,47.969263,-1.90,5.2,250,60,764.18,Запах гари 3
1,2020-02-14 07:39:20+00,46.335238,48.115489,-1.99,9.6,272,70,768.00,Химический


In [33]:
m = folium.Map(location=[46.35, 48.05], zoom_start=12)

for index, row in data.iterrows():
    folium.CircleMarker([row['lat'], row['long']],
                        radius=2,
                        popup=row['description'] + row['date%time'] + 'Напрваление ветра: ' + str(row['winddir']) + ' град.',
                        fill_color="#3db7e4", 
                       ).add_to(m)
fog = data[['lat', 'long']]
    
m.add_child(plugins.HeatMap(fog, radius=15))
m.save('map.html')

In [34]:
ftp = ftplib.FTP(host, ftp_user, ftp_password)

file = 'map.html'
file_to_upload = open(file, 'rb')
ftp.storbinary('STOR www/data-lab.kremppi.com/' + file, file_to_upload)

file = 'log.csv'
file_to_upload = open(file, 'rb')
ftp.storbinary('STOR www/data-lab.kremppi.com/' + file, file_to_upload)

'226 Transfer complete'